A notebook for testing some decorator concepts

## Two decorators that with arguments affecting the arguments

In [78]:
from functools import wraps
from inspect import getfullargspec, signature

In [60]:
def mul(multiplier):
    def decorator(func):
        @wraps(func)
        def wrapper(value):
            value = value * multiplier
            return func(value)
        return wrapper
    return decorator

def add(adder):
    def decorator(func):
        @wraps(func)
        def wrapper(value):
            value = value + adder
            return func(value)
        return wrapper
    return decorator

In [61]:
@mul(2)
@add(3)
def muladd(value):
    return value
muladd(5)


13

In [62]:
@add(3)
@mul(2)
def addmul(value):
    return value

addmul(5)

16


## Two decorators with arguments checking the arguments
Notice the arguments change across functions

In [76]:
from functools import wraps

def mul(param, multiplier):
    def decorator(func):
        @wraps(func)
        def wrapper(*argsmul, **kwargs):
            print(f"Mul - sig - {signature(func)}")
            print(f"Mul - sigparam - {signature(func).parameters}")
            print(f"Mul - argspec - {getfullargspec(func)}")
            print(f"Mul - argskwargs - {argsmul} - {kwargs}")
            return func(*argsmul, **kwargs)
            print("Mul - Return")
        return wrapper
    return decorator

def add(param, adder):
    def decorator(func):
        @wraps(func)
        def wrapper(*argsadd, **kwargs):
            print(f"Add - sig - {signature(func)}")
            print(f"Add - sigparam - {signature(func).parameters}")
            print(f"Add - argspec - {getfullargspec(func)}")
            print(f"Add - argskwargs - {argsadd} - {kwargs}")
            return func(*argsadd, **kwargs)
            print(f"Add - return")
        return wrapper

    return decorator

In [77]:
@mul('value', 3)
@add('value', 4)
def func2(value=10):
    "docstring"
    return value

func2()

Mul - sig - (value=10)
Mul - sigparam - OrderedDict([('value', <Parameter "value=10">)])
Mul - argspec - FullArgSpec(args=[], varargs='argsadd', varkw='kwargs', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})
Mul - argskwargs - () - {}
Add - sig - (value=10)
Add - sigparam - OrderedDict([('value', <Parameter "value=10">)])
Add - argspec - FullArgSpec(args=['value'], varargs=None, varkw=None, defaults=(10,), kwonlyargs=[], kwonlydefaults=None, annotations={})
Add - argskwargs - () - {}


10

The  makefun library can be used to resolve this in a way that functools does not

- https://stackoverflow.com/questions/308999/what-does-functools-wraps-do/55102697#55102697
- https://stackoverflow.com/questions/33190518/how-can-i-pass-arguments-to-decorator-process-there-and-forward-to-decorated-f 

In [72]:
import makefun 
@mul('value', 3)
@add('value', 4)
def func2(value=10):
    return value

func2()

Mul - sig - (value=10)
Mul - sigparam - OrderedDict([('value', <Parameter "value=10">)])
Mul - argspec - FullArgSpec(args=['value'], varargs=None, varkw=None, defaults=(10,), kwonlyargs=[], kwonlydefaults=None, annotations={})
Mul - argskwargs - () - {'value': 10}
Add - sig - (value=10)
Add - sigparam - OrderedDict([('value', <Parameter "value=10">)])
Add - argspec - FullArgSpec(args=['value'], varargs=None, varkw=None, defaults=(10,), kwonlyargs=[], kwonlydefaults=None, annotations={})
Add - argskwargs - () - {'value': 10}


10

In [44]:
def mul(param, multiplier):
    def decorator(func):
        @wraps(func)
        def wrapper(*argsmul, **kwargs):
            print(f"Mul - sig - {signature(func)}")
            print(f"Mul - sigparam - {signature(func).parameters}")
            print(f"Mul - argspec - {getfullargspec(func)}")
            print(f"Mul - argskwargs - {argsmul} - {kwargs}")
            return func(*argsmul, **kwargs)
            print("Mul - Return")
        return wrapper
    return decorator

def add(param, adder):
    def decorator(func):
        @wraps(func)
        def wrapper(*argsadd, **kwargs):
            print(f"Add - sig - {signature(func)}")
            print(f"Add - sigparam - {signature(func).parameters}")
            print(f"Add - argspec - {getfullargspec(func)}")
            print(f"Add - argskwargs - {argsadd} - {kwargs}")
            return func(*argsadd, **kwargs)
            print(f"Add - return")
        return wrapper

    return decorator

In [73]:
def check_arg_positive(*params):
    """
    Checks the arg value is positive

    Usage:
    >>> @check_arg_positive('value')
    ... def func(value=-10, other=-6):
    ...     return value

    >>> func(10)
    10

    >>> func()
    Traceback (most recent call last):
        ...
    ValueError: -10 is not positive

    >>> func(-8)
    Traceback (most recent call last):
        ...
    ValueError: -8 is not positive
    """

    def inner(func):
        @makefun.wraps(func)
        def wrapper(*args, **kwargs):
            for param in params:
                # Get the value using one of 3 methods
                if param in getfullargspec(func)[0]:
                    # Check kwargs - Is the param in the kwarg
                    if param in kwargs:
                        value = kwargs[param]
                    else:
                        param_idx = getfullargspec(func)[0].index(param)
                        # Check args - is the param in the arg
                        if param_idx < len(args):
                            value = args[param_idx]
                        # Check default - is the param a default value
                        else:
                            value = signature(func).parameters[param].default

                # Raise an error if it is negative
                if value < 0:
                    raise ValueError(f"{value} is not positive")

            return func(*args, **kwargs)

        return wrapper

    return inner

In [85]:
def check_arg_type(func):
    """
    Checks the args match the input type annotations

    Usage:
    >>> @check_arg_type
    ... def func(x: int, y: str, z):
    ...     return (x, y, z)

    >>> func(3.0, 2, 1)
    Traceback (most recent call last):
        ...
    TypeError: 3.0 is not of type <class 'int'>

    >>> func(3, 2, 1)
    Traceback (most recent call last):
        ...
    TypeError: 2 is not of type <class 'str'>

    >>> func(3, '2', 1)
    (3, '2', 1)
    """

    @wraps(func)
    def wrapper(*args):
        for index, arg in enumerate(getfullargspec(func)[0]):
            if arg in func.__annotations__:
                if not isinstance(args[index], func.__annotations__[arg]):
                    raise TypeError(
                        f"{args[index]} is not of type {func.__annotations__[arg]}"
                    )

        return func(*args)

    return wrapper

In [86]:
def coerce_arg_type(func):
    """
    Coerces the args to match the type annotations

    Usage:
    >>> @coerce_arg_type
    ... def func(x: int, y: str, z):
    ...     return (x, y, z)

    >>> func(3.0, 2, 1)
    (3, '2', 1)

    >>> func(3, '2', 1)
    (3, '2', 1)
    """

    @wraps(func)
    def wrapper(*args, **kwargs):
        args = list(args)
        for index, arg in enumerate(getfullargspec(func)[0]):
            if arg in func.__annotations__:
                args[index] = func.__annotations__[arg](args[index])
        return func(*tuple(args), **kwargs)

    return wrapper

In [93]:
@coerce_arg_type
@check_arg_type
@check_arg_positive("value")
def func2(value:int):
    return value

func2(-10)

ValueError: -10 is not positive